In [1]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Constructing a LangGraphAgent

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/agents/genai-experience-concierge/langgraph-demo/backend/notebooks/langgraph-agent.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fagents%2Fgenai-experience-concierge%2Flanggraph-demo%2Fbackend%2Fnotebooks%2Flanggraph-agent.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/agents/genai-experience-concierge/langgraph-demo/backend/notebooks/langgraph-agent.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agents/genai-experience-concierge/langgraph-demo/backend/notebooks/langgraph-agent.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agents/genai-experience-concierge/langgraph-demo/backend/notebooks/langgraph-agent.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agents/genai-experience-concierge/langgraph-demo/backend/notebooks/langgraph-agent.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agents/genai-experience-concierge/langgraph-demo/backend/notebooks/langgraph-agent.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agents/genai-experience-concierge/langgraph-demo/backend/notebooks/langgraph-agent.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agents/genai-experience-concierge/langgraph-demo/backend/notebooks/langgraph-agent.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| | |
|-|-|
|Author(s) | [Pablo Gaeta](https://github.com/pablofgaeta) |

## Overview

This notebook demonstrates the necessary code to transform any LangGraph graph into a `LangGraphAgent` using the `concierge.langgraph_server` module.

This is a useful interactive way to develop agents before deploying them on a server.

## Import dependencies

In [2]:
import asyncio
from typing import TypedDict
import uuid

from concierge.langgraph_server import langgraph_agent, schemas
from langgraph import graph, types

## Define graph state schema and simple graph implementation

In [3]:
# State schema


class State(TypedDict, total=False):
    id: str


# Node functions


async def set_id(state: State):
    new_id = state.get("id")
    assert new_id is not None, "must set ID"

    await asyncio.sleep(1)
    return types.Command(update=State(id=new_id), goto="REVERSE_ID")


async def reverse_id(state: State):
    new_id = state.get("id")
    assert new_id is not None, "ID must be set before reversing"

    await asyncio.sleep(1)
    return types.Command(update=State(id=new_id[::-1]))


# Graph

state_graph = graph.StateGraph(state_schema=State)
state_graph.add_node("SET_ID", set_id)
state_graph.add_node("REVERSE_ID", reverse_id)
state_graph.set_entry_point("SET_ID");

## Build a _stateful_ agent

In [4]:
basic_agent = langgraph_agent.LangGraphAgent(
    state_graph=state_graph,
    checkpointer_config=schemas.MemoryBackendConfig(),
)

**Test**: Run a query in debug stream mode

In [5]:
async for chunk in basic_agent.stream(
    input=State(id="hello"),
    config={"configurable": {"thread_id": uuid.uuid4().hex}},
    stream_mode="debug",
):
    print(chunk)

**Test**: Run a query in values mode (full state returned each time)

In [6]:
async for chunk in basic_agent.stream(
    input=State(id="hello"),
    config={"configurable": {"thread_id": uuid.uuid4().hex}},
    stream_mode="values",
):
    print("complete state", "=", chunk)

**Test**: Run a query in updates mode (only state updates returned by each node)

In [7]:
async for chunk in basic_agent.stream(
    input=State(id="hello"),
    config={"configurable": {"thread_id": uuid.uuid4().hex}},
    stream_mode="updates",
):
    print("updates", "->", chunk)

**Test**: Run a query with a list of stream modes (yields stream mode and chunk data)

In [8]:
async for stream_mode, chunk in basic_agent.stream(
    input=State(id="hello"),
    config={"configurable": {"thread_id": uuid.uuid4().hex}},
    stream_mode=["updates"],
):
    print(stream_mode, "->", chunk)

**Test**: Attempt a stateless query, fails since checkpointer requires thread ID. To run stateless queries, must build a stateless agent like the example in the next section.

In [9]:
async for stream_mode, chunk in basic_agent.stream(
    input=State(id="hello"),
    config={"configurable": {"thread_id": uuid.uuid4().hex}},
    stream_mode=["updates"],
):
    print(stream_mode, "->", chunk)

## Build a _stateless_ agent

In [10]:
stateless_basic_agent = langgraph_agent.LangGraphAgent(
    state_graph=state_graph,
    checkpointer_config=None,  # no checkpointer means it will only work with stateless requests
)

**Test**: Run stateless run without threads

In [11]:
async for chunk in stateless_basic_agent.stream(
    input=State(id="hello"),
    stream_mode="updates",
):
    print("complete state", "=", chunk)